# Output 문장 구조 EDA

In [11]:
!pip install nltk

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 660.6 kB/s eta 0:00:03
   ----------- ---------------------------- 0.4/1.5 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------  1.5/1.5 MB 13.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 12.0 MB/s eta 0:00:00


In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from transformers import AutoTokenizer
from collections import defaultdict
from nltk import ngrams

In [2]:
def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    try:
        df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output','inference']]

    except:
        df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

In [8]:
# train_df = make_dataframe('../resource/data/일상대화요약_train.json')
# dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
# test_df = make_dataframe('../resource/data/일상대화요약_test.json')
train_df = make_dataframe('./train.json')
dev_df = make_dataframe('./dev.json')
test_df = make_dataframe('./test.json')

In [73]:
nova_result = make_dataframe('../results/hypernova.json')
dev_nova_result = make_dataframe('../results/hypernova_dev.json')
hypernova_result = make_dataframe('../results/hypernova.json')
supernova_reuslt = make_dataframe('../results/supernova3.json')

## 전반적인 요약 유형


- 맨 앞 전반적인 요약 부분은 어떤 유형의 문장이 있는지 확인

In [9]:
from nltk.util import ngrams

def find_common_ngrams(df, n):

    sentences = df['output'].apply(lambda x: x.split('.')[0])

    def preprocess(sentence):
        return sentence.split()

    ngrams_list = []
    total_ngrams_set = set()
    total_ngrams_dict = defaultdict(int)
    
    for sentence in sentences:
        tokens = preprocess(sentence)
        ngrams_list.append(list(ngrams(tokens, n)))
    

    # 같은 n-gram이 나타나는 문장을 찾기
    for i in range(len(ngrams_list)-1):
        common_ngrams = set(ngrams_list[i])
        
        for ngrams_set in ngrams_list[i+1:]:
            total_ngrams_set.update(common_ngrams.intersection(ngrams_set))

    # n-gram이 나타나는 문장의 개수 세기
    for common_ngram in total_ngrams_set:
        for ngrams_set in ngrams_list:
            if common_ngram in ngrams_set:
                total_ngrams_dict[common_ngram] += 1

    # 정렬
    total_ngrams_dict = dict(sorted(total_ngrams_dict.items(), key=lambda x: x[1], reverse=True))
    
    return total_ngrams_dict

In [10]:
common_ngrams = find_common_ngrams(train_df, 4)
# for ngram in common_ngrams:
#     print(' '.join(ngram))
common_ngrams

{('두', '화자는', '이', '대화에서'): 152,
 ('이', '대화에서', '두', '화자는'): 7,
 ('이', '대화에서', '화자들은', '좋아하는'): 7,
 ('화자는', '이', '대화에서', '좋아하는'): 6,
 ('이', '대화에서', '화자들은', '기억에'): 4,
 ('화자는', '이', '대화에서', '기억에'): 4,
 ('이', '대화에서', '기억에', '남는'): 4,
 ('대화에서', '화자들은', '기억에', '남는'): 4,
 ('이', '대화에서', 'SD2100585와', 'SD2100586은'): 4,
 ('화자는', '이', '대화에서', '건강'): 4,
 ('이', '대화에서', '인상', '깊었던'): 3,
 ('두', '사람은', '기억에', '남는'): 3,
 ('화자는', '이', '대화에서', '자신의'): 3,
 ('이', '대화에서', '화자들은', '요즘'): 3,
 ('대화에서', '화자', 'SD2100517과', 'SD2100518은'): 3,
 ('화자는', '이', '대화에서', '자녀의'): 3,
 ('이', '대화에서', '화자들은', '운동에'): 3,
 ('화자는', '이', '대화에서', '영화'): 3,
 ('음식에', '관한', '대화를', '나눴습니다'): 3,
 ('이', '대화에서', '화자들은', '선호하는'): 3,
 ('싶은', '선물에', '대해', '말했습니다'): 3,
 ('이', '대화에서', '화자', 'SD2100517과'): 3,
 ('가고', '싶은', '여행지에', '대해'): 3,
 ('받고', '싶은', '선물에', '대해'): 3,
 ('화자들은', '운동에', '대해', '이야기했습니다'): 3,
 ('이', '대화에서', '화자들은', '건강'): 3,
 ('화자는', '이', '대화에서', '아르바이트'): 3,
 ('대화에서', '화자들은', '운동에', '대해'): 3,
 ('화자는', '이', '대화에서', '인상'): 3,

In [11]:
common_ngrams = find_common_ngrams(dev_df, 4)
common_ngrams

{('두', '화자는', '이', '대화에서'): 54,
 ('이', '대화에서', '두', '화자는'): 8,
 ('화자는', '이', '대화에서', '좋아하는'): 5,
 ('이', '대화에서', '화자들은', '좋아하는'): 4,
 ('두', '사람은', '이', '대화에서'): 4,
 ('화자는', '이', '대화에서', '건강'): 4,
 ('이', '대화에서', '선물에', '대해'): 3,
 ('화자는', '이', '대화에서', '학교생활에'): 3,
 ('대화에서', '선물에', '대해', '말했습니다'): 3,
 ('화자는', '이', '대화에서', '학교'): 3,
 ('화자는', '이', '대화에서', '선물에'): 3,
 ('대화에서', '학교', '생활에', '대해'): 2,
 ('학교', '생활에', '대해', '말했습니다'): 2,
 ('관리와', '운동에', '대해', '말했습니다'): 2,
 ('이', '대화에서', '학교생활에', '대해'): 2,
 ('이', '대화에서', '화자들은', '가족'): 2,
 ('아르바이트에', '대해', '이야기를', '나눴습니다'): 2,
 ('화자는', '이', '대화에서', '결혼과'): 2,
 ('이', '대화에서', '학교', '생활에'): 2,
 ('화자는', '이', '대화에서', '현재'): 2,
 ('대화에서', '학교생활에', '대해', '말했습니다'): 2,
 ('대화에서', '두', '화자는', '좋아하는'): 2}

In [12]:
import re

def find_another_type_total_summary(df):
    types = ["두 화자는 이 대화에서", "두 화자는", "화자들은" ,"두 사람은", "이 대화에서는"]
    types2 = r"SD\d{7}(?:와|과).*SD\d{7}(?:은|는)"# r".+[과와].+[은는]"
    cnt = 0

    sentences = df['output'].apply(lambda x: x.split('.')[0] if x.split('.')[1] else print(x))

    for sentence in sentences:
        yes = 0
        for type in types:
            if type in sentence:
                cnt += 1
                yes = 1
                break
            elif re.search(types2, sentence):
                cnt += 1
                yes = 1
                break

        if yes==0:

            print(sentence)

    print(cnt)

In [153]:
find_another_type_total_summary(train_df)

이 대화에서 화자 SD2000875는 진로에 대한 고민을 이야기했고 화자SD2000873은 공감과 조언을 통한 상담을 해주었습니다
SD2100525는 가끔씩 밖에 나가거나 비가 온 후 냄새가 좋아서 자주 나가야겠다는 생각을 했으며 땀이 날 정도로 운동을 해야 한다는 의사 선생님의 말씀을 듣고 한동안 열심히 운동을 했다가 요즘 뱃살이 많아져서 너무 스트레스를 받는다고 이야기했습니다
504


In [157]:
find_another_type_total_summary(dev_df)

이 대화에서 가장 인상 깊었던 여행지와 추천하고 싶은 여행지를 공유했습니다
SD2000862는 엔시티의 보스 착장 중 제복 착장이 멤버들의 피지컬 덕분에 좋았고 터프하게 랩을 하는 마크의 모습이 인상깊었다고 말했습니다
100


## 유형은 총 6가지

- "두 화자는 이 대화에서"
- "두 화자는"
- "화자들은"
- "두 사람은"
- "이 대화에서는"
- r".+[과와].+[은는]"

- 본 유형에 해당하지 않는 샘플이 train에서 2개, dev에서 2개 존재
    - 이 경우는 같은 speaker에 해당하는 샘플(주변 샘플)에서 '전반적인 요약'에 해당하는 부분을 온전히 얻어올 수 있었음
    - 또는 간단한 수정을 통해 해결 가능 (이 대화에서 -> 두 화자는 이 대화에서)

---

- **train-000032**
    - 이 대화에서 화자 SD2000875는 진로에 대한 고민을 이야기했고 화자SD2000873은 공감과 조언을 통한 상담을 해주었습니다 \
        -> `"두 화자는 이 대화에서 진로 관련 고민에 대해 이야기했습니다." 로 변경`

<br/>

- **train-000418**
    - 전반적인 요약이 존재하지 않아 -> 이전 샘플의 전반적인 요약을 그대로 가져옴\
        -> `"두 화자는 이 대화에서 다이어트에 대해 이야기했습니다." 를 맨 앞 문장에 추가.` 이어서 다음 문장을 붙임
    - SD2100525는 가끔씩 밖에 나가거나 비가 온 후 냄새가 좋아서 자주 나가야겠다는 생각을 했으며 땀이 날 정도로 운동을 해야 한다는 의사 선생님의 말씀을 듣고 한동안 열심히 운동을 했다가 요즘 뱃살이 많아져서 너무 스트레스를 받는다고 이야기했습니다

<br/>

- **dev-000074**
    - `이 대화에서 -> 두 화자는 이 대화에서`

- **dev-000093**
    - 전반적인 요약이 존재하지 않아
        - `"두 화자는 이 대화에서 엔시티와 방탄소년단에 대해 이야기 했습니다." 를 맨 앞 문장에 추가.`
    - SD2000862는 엔시티의 보스 착장 중 제복 착장이 멤버들의 피지컬 덕분에 좋았고 터프하게 랩을 하는 마크의 모습이 인상깊었다고 말했습니다

---

In [158]:
# 변경 이후
train_df = make_dataframe('./train.json')
dev_df = make_dataframe('./dev.json')
test_df = make_dataframe('./test.json')

In [159]:
find_another_type_total_summary(train_df)

506


In [160]:
find_another_type_total_summary(dev_df)

102


## 전반적인 요약 형식 통일

- 일단, 모든 경우 중에
    - "두 화자는 이 대화에서" 이다
        - train(150 + 7) / 506  
        - dev(52 + 8) / 102

    - 모든 경우를 "두 화자는 이 대화에서" 방식으로 통일시켜보자
    - 그렇다면, 모델은 '전반적인 요약'을 어떤 유형으로 답해야하는지 맞추는 것이 아닌

        - 그 뒤에 해당하는 '전반적인 요약'에 해당하는 작업을 온전하게 집중할 수 있을 것이다

## type1과 type2에 해당하는 경우 뒤에 SD가 들어간 sepeaker가 있는지 확인

In [161]:
# type1과 type2에 해당하는 경우 뒤에 SD가 들어간 sepeaker가 있는지 확인

def find_SD_after_type(df):
    types = ["두 화자는 이 대화에서", "두 화자는", "화자들은" ,"두 사람은", "이 대화에서는"]
    types2 = r"SD\d{7}(?:와|과).*SD\d{7}(?:은|는)"
    cnt = 0

    sentences = df['output'].apply(lambda x: x.split('.')[0])

    for sentence in sentences:
        for type in types:
            if type in sentence:
                match = re.search(type + r'(.*)', sentence, re.DOTALL)
                post_sentence = match.group(1).strip() 
                if 'SD' in sentence:
                    cnt += 1
                    print('post:',post_sentence)
                    print('total:',sentence)
                    print('-'*30)
                    break
            elif re.search(types2, sentence):
                match = re.search(types2 + r'(.*)', sentence, re.DOTALL)
                post_sentence = match.group(1).strip() 
                if 'SD' in post_sentence:
                    cnt += 1
                    print('post:',post_sentence)
                    print('total',sentence)
                    print('-'*30)
                    break

    print(cnt)

In [162]:
find_SD_after_type(train_df)

post: SD2000039의 꿈인 전원주택에서 살기 위한 준비 과정에 대해 이야기했습니다
total: 두 화자는 이 대화에서 SD2000039의 꿈인 전원주택에서 살기 위한 준비 과정에 대해 이야기했습니다
------------------------------
post: 애완동물을 키울 때 장단점에 대해 말했습니다 SD2001335는 반려동물 기르는 것을 선호하지 않았지만, 자녀가 너무 원해서 고양이를 기르기 시작했고 1년 정도 됐다고 말했습니다
total: 두 화자는 이 대화에서 애완동물을 키울 때 장단점에 대해 말했습니다 SD2001335는 반려동물 기르는 것을 선호하지 않았지만, 자녀가 너무 원해서 고양이를 기르기 시작했고 1년 정도 됐다고 말했습니다
------------------------------
2


In [163]:
find_SD_after_type(dev_df)

0


#### 결과
- 506개의 train 중에 단 1개, dev는 존재하지 않음
- 그러므로, 데이터에서
    - 맨 앞 문장의 경우
    - SD~~ 와 같은 speaker가 등장하는 경우는
        - 문장 type을 맞추기 위한 경우를 제외하고는 없다
        - 단 하나의 예외가 있을 뿐
    
    - 예외 처리를 위해 "SD2000039의 꿈인"을 없애주자

- **train-000020**

    - post: SD2000039의 꿈인 전원주택에서 살기 위한 준비 과정에 대해 이야기했습니다
    - total: 두 화자는 이 대화에서 SD2000039의 꿈인 전원주택에서 살기 위한 준비 과정에 대해 이야기했습니다

        -> `"두 화자는 이 대화에서 전원주택에서 살기 위한 준비 과정에 대해 이야기했습니다" 로 변경`


<br/>

- **train-000176**

    - post: 애완동물을 키울 때 장단점에 대해 말했습니다 SD2001335는 반려동물 기르는 것을 선호하지 않았지만, 자녀가 너무 원해서 고양이를 기르기 시작했고 1년 정도 됐다고 말했습니다
    - total: 두 화자는 이 대화에서 애완동물을 키울 때 장단점에 대해 말했습니다 SD2001335는 반려동물 기르는 것을 선호하지 않았지만, 자녀가 너무 원해서 고양이를 기르기 시작했고 1년 정도 됐다고 말했습니다

    - **이 경우는 맨 앞 문장에 해당하는 '전반적인 요약' 문장 맨 끝에 '.'이 없는 이상 샘플이다**

        -> `'두 화자는 이 대화에서 애완동물을 키울 때 장단점에 대해 말했습니다' 뒤에 '.'를 추가`

- 변경 이후

In [164]:
# 변경 이후
train_df = make_dataframe('./train.json')
dev_df = make_dataframe('./dev.json')
test_df = make_dataframe('./test.json')

find_another_type_total_summary(train_df)
find_another_type_total_summary(dev_df)

find_SD_after_type(train_df)
find_SD_after_type(dev_df)

506
102
0
0


## type1과 type2의 앞에 있는 내용 확인

In [175]:
# type1과 type2을 포함하여 앞에 있는 문장 종류를 확인

def find_before_type(df):
    types = ["두 화자는 이 대화에서", "두 화자는", "화자들은" ,"두 사람은", "이 대화에서는"]
    types2 = r"SD\d{7}(?:와|과).*SD\d{7}(?:은|는)"
    cnt = 0

    sentences = df['output'].apply(lambda x: x.split('.')[0])

    for idx, sentence in enumerate(sentences):
        yes = 0
        if re.search(types2, sentence):
            match = re.search(r'(.*)'+types2, sentence, re.DOTALL)
            pre_sentence = match.group(0).strip() 
            print(f'{idx} pre:',pre_sentence)
            # print('total',sentence)
            print('-'*30)
            yes = 1
            continue
        for type in types:
            if type in sentence:
                match = re.search(r'(.*)'+type, sentence, re.DOTALL)
                pre_sentence = match.group(0).strip() 
                print(f'{idx} pre:',pre_sentence)
                # print('total:',sentence)
                print('-'*30)
                yes = 1
                break
        
        if yes==0:
            raise ValueError('No type detected')

In [176]:
find_before_type(train_df)

0 pre: 이 대화에서 화자들은
------------------------------
1 pre: 이 대화에서 화자들은
------------------------------
2 pre: 이 대화에서 화자들은
------------------------------
3 pre: 두 화자는 이 대화에서
------------------------------
4 pre: 두 화자는 이 대화에서
------------------------------
5 pre: 두 화자는 이 대화에서
------------------------------
6 pre: 이 대화에서 화자들은
------------------------------
7 pre: 이 대화에서 화자들은
------------------------------
8 pre: 이 대화에서 화자들은
------------------------------
9 pre: 이 대화에서 화자들은
------------------------------
10 pre: 이 대화에서 화자들은
------------------------------
11 pre: 이 대화에서 화자들은
------------------------------
12 pre: 이 대화에서 화자들은
------------------------------
13 pre: 이 대화에서 화자 SD2000034와 화자 SD2000035는
------------------------------
14 pre: SD2000038과 SD2000039는
------------------------------
15 pre: SD2000038과 SD2000039는
------------------------------
16 pre: SD2000038과 SD2000039는
------------------------------
17 pre: 두 화자는 이 대화에서
------------------------------
18 pre: 두 화자는 이 대화에서
--------------

In [177]:
find_before_type(dev_df)

0 pre: 두 화자는 이 대화에서
------------------------------
1 pre: 두 화자는 이 대화에서
------------------------------
2 pre: 두 화자는 이 대화에서
------------------------------
3 pre: 두 사람은
------------------------------
4 pre: 두 사람은
------------------------------
5 pre: 두 화자는 이 대화에서
------------------------------
6 pre: 두 화자는 이 대화에서
------------------------------
7 pre: 두 화자는 이 대화에서
------------------------------
8 pre: 두 화자는 이 대화에서
------------------------------
9 pre: 두 사람은
------------------------------
10 pre: 두 사람은
------------------------------
11 pre: 두 화자는 이 대화에서
------------------------------
12 pre: 두 화자는 이 대화에서
------------------------------
13 pre: 이 대화에서 두 화자는
------------------------------
14 pre: 이 대화에서 두 화자는
------------------------------
15 pre: 이 대화에서 두 화자는
------------------------------
16 pre: 이 대화에서 두 화자는
------------------------------
17 pre: 두 화자는 이 대화에서
------------------------------
18 pre: 두 화자는 이 대화에서
------------------------------
19 pre: 두 화자는 이 대화에서
-----------------------------

### 확인 결과

- type 앞에 특별하게 요약과 관련된 내용 존재 x
- 즉, 현재 type을 적절하게 잘 뽑아낸 것이라 볼 수 있음
- 모델이 이 여러가지 타입을 맞추는 것이 아닌
- 핵심에 해당하는 "전반적인 요약"에 집중할 수 있게끔
- type을 가장 보편적인 `두 화자는 이 대화에서` 하나로 통일

### 통일 이후

In [15]:
train_df = make_dataframe('./train.json')
dev_df = make_dataframe('./dev.json')
test_df = make_dataframe('./test.json')

In [16]:
find_common_ngrams(train_df, 4)

{('두', '화자는', '이', '대화에서'): 506,
 ('화자는', '이', '대화에서', '좋아하는'): 23,
 ('화자는', '이', '대화에서', '기억에'): 13,
 ('이', '대화에서', '기억에', '남는'): 13,
 ('화자는', '이', '대화에서', '건강'): 12,
 ('화자는', '이', '대화에서', '여행'): 9,
 ('화자는', '이', '대화에서', '선호하는'): 9,
 ('화자는', '이', '대화에서', '자신의'): 7,
 ('화자는', '이', '대화에서', '가족'): 7,
 ('화자는', '이', '대화에서', '영화'): 5,
 ('화자는', '이', '대화에서', '운동에'): 5,
 ('이', '대화에서', '가고', '싶은'): 5,
 ('화자는', '이', '대화에서', '아르바이트'): 5,
 ('화자는', '이', '대화에서', '다이어트'): 5,
 ('화자는', '이', '대화에서', '가고'): 5,
 ('화자는', '이', '대화에서', '운동과'): 5,
 ('화자는', '이', '대화에서', '요즘'): 5,
 ('화자는', '이', '대화에서', '자녀의'): 4,
 ('화자는', '이', '대화에서', '결혼과'): 4,
 ('이', '대화에서', '건강', '관리에'): 4,
 ('화자는', '이', '대화에서', '최근'): 4,
 ('이', '대화에서', '운동에', '대해'): 4,
 ('화자는', '이', '대화에서', '가족에'): 4,
 ('화자는', '이', '대화에서', '키우고'): 4,
 ('화자는', '이', '대화에서', '건강을'): 4,
 ('화자는', '이', '대화에서', '자주'): 4,
 ('화자는', '이', '대화에서', '결혼에'): 4,
 ('화자는', '이', '대화에서', '대화에서'): 4,
 ('화자는', '이', '대화에서', '선물에'): 4,
 ('이', '대화에서', '인상', '깊었던'): 3,
 ('화자는', '이', 

In [17]:
find_common_ngrams(dev_df, 4)

{('두', '화자는', '이', '대화에서'): 102,
 ('화자는', '이', '대화에서', '좋아하는'): 12,
 ('화자는', '이', '대화에서', '건강'): 5,
 ('화자는', '이', '대화에서', '이'): 4,
 ('이', '대화에서', '이', '대화에서'): 4,
 ('이', '대화에서', '선물에', '대해'): 3,
 ('화자는', '이', '대화에서', '학교생활에'): 3,
 ('대화에서', '선물에', '대해', '말했습니다'): 3,
 ('화자는', '이', '대화에서', '학교'): 3,
 ('화자는', '이', '대화에서', '선물에'): 3,
 ('화자는', '이', '대화에서', '가족'): 2,
 ('이', '대화에서', '건강', '관리를'): 2,
 ('대화에서', '학교', '생활에', '대해'): 2,
 ('화자는', '이', '대화에서', '가장'): 2,
 ('학교', '생활에', '대해', '말했습니다'): 2,
 ('관리와', '운동에', '대해', '말했습니다'): 2,
 ('이', '대화에서', '학교생활에', '대해'): 2,
 ('이', '대화에서', '연애와', '결혼에'): 2,
 ('아르바이트에', '대해', '이야기를', '나눴습니다'): 2,
 ('화자는', '이', '대화에서', '결혼과'): 2,
 ('이', '대화에서', '학교', '생활에'): 2,
 ('화자는', '이', '대화에서', '부모와'): 2,
 ('화자는', '이', '대화에서', '현재'): 2,
 ('대화에서', '학교생활에', '대해', '말했습니다'): 2}